## Install and import any required Python libraries.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Ignore warnings within the notebook
import warnings
warnings.filterwarnings('ignore')


## Query the database to extract these for each user:

In [ ]:
-- Query the database to extract these for each user:
    -- their user ID
    -- their age
    -- their total amount spent
-- However, exclude the following:
    -- users where a user ID is missing
    -- orders that are returned or canceled 
SELECT 
    U.USER_ID, 
    U.AGE, 
    SUM(I.SALE_PRICE) AS TOTAL_SPENT
FROM 
    USERS U
INNER JOIN
    ORDERS O
ON 
    U.USER_ID = O.USER_ID
INNER JOIN
    ORDER_ITEMS I
ON
    O.ORDER_ID = I.ORDER_ID
WHERE
    U.USER_ID IS NOT NULL
AND
    O.STATUS != 'Returned'
AND
    O.STATUS != 'Cancelled'
GROUP BY
    U.USER_ID,
    U.AGE
ORDER BY
    TOTAL_SPENT DESC
LIMIT 3;

## Store the query results in a pandas dataframe

In [ ]:
# Store the query results in a pandas dataframe
# Referencing the cell name user_details
df_users = user_details.to_pandas()

## Perform the following exploratory steps:

In [ ]:
# Calculate the dimensions of the dataframe.
df_users.shape

In [ ]:
# Identify the datatypes for each field.
df_users.info()

In [ ]:
# Preview a few records of data.
df_users.head()

In [ ]:
# Determine if there are any null values.
df_users.isnull().sum()

In [ ]:
# Determine if there are any duplicates.
sum(df_users.duplicated(df_users.columns))

In [ ]:
# Generate descriptive stats for each field.
df_users.describe()

## Upload a histogram showing the distribution of ages.

In [ ]:
# Upload a histogram showing the distribution of ages.
sns.histplot(data = df_users, x="AGE", discrete=True)
plt.xlim(11, 71)
plt.ylim(1000, 1250)
plt.show()

## Upload a histogram showing the distribution of amounts spent.

In [ ]:
# Upload a histogram showing the distribution of amounts spent.
sns.histplot(data = df_users, x="TOTAL_SPENT")
plt.xlim(0, 2000)

## Scale the input data prior to clustering.

In [ ]:
# Scale the input data prior to clustering.
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_users[['AGE', 'TOTAL_SPENT']])

# Preview the scaled data (numpy array)
scaled_data[:5]

## Perform k-means clustering.

In [ ]:
# Perform k-means clustering.
# Select the # of clusters based on the stakeholder interview = 4.
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(scaled_data)

In [ ]:
# Looking into kmeans a little bit
# Display the cluster centers.
kmeans.cluster_centers_

In [ ]:
# Display the cluster assignments for a few rows of the data.
kmeans.labels_[:5]

## Add cluster assignments as a new field in the dataframe.

In [ ]:
# Add cluster assignments as a new field in the dataframe.
df_users['CLUSTER'] = kmeans.labels_

# Preview the updated data.
df_users.head()

## Upload a plot that counts the # of customers in each cluster.

In [ ]:
# Upload a plot that counts the # of customers in each cluster
sns.countplot(x='CLUSTER', hue='CLUSTER', palette='Dark2_r', data=df_users)

## Upload a heatmap showing customer ages and amounts with clusters as the colors.

In [ ]:
# Upload a heatmap showing customer ages and amounts with clusters as the colors.
sns.displot(df_users, x='AGE',  y='TOTAL_SPENT', hue="CLUSTER", palette="Dark2_r")
plt.show()

## Upload a boxplot showing the distribution of amounts by cluster.

In [ ]:
# Upload a boxplot showing the distribution of amounts by cluster.
sns.boxplot(x="CLUSTER", y="TOTAL_SPENT", hue="CLUSTER", palette="Dark2_r", data=df_users)
plt.show()